<h1>ニューラルネットワーク</h1>

In [1]:
import torch
import torch.nn as nn  #ミニバッチが前提？
import torch.nn.functional as F

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 入力チャンネル : 1 1Sample?
        # 出力チャンネル : 6 6Channels?
        # 5 * 5 正方形の畳み込み層  5 Height 5 Width 
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        # アフィンオペレーション
        #  y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # 上は同じ意味（正方形は2などで良い）
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


<h4>forward関数を定義するだけで自動的にbackward関数が定義される</h4>

In [8]:
## 学習可能なモデルのパラメータはnet.parameters()で取得
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [10]:
input = torch.randn(1,1,32,32)
out = net(input)
print(input)
print(input.size())
print(out)

tensor([[[[-0.4881, -1.1541, -1.3453,  ...,  1.7916,  0.5993,  0.0182],
          [ 1.5172, -0.5187,  0.4842,  ...,  1.0249, -0.1141,  3.1099],
          [ 0.3610, -0.0271, -0.8338,  ..., -2.2554,  0.8961,  0.7331],
          ...,
          [ 0.7301, -1.4158,  1.5179,  ...,  0.2344,  0.8844,  0.5803],
          [ 1.0194,  0.6932, -0.3603,  ...,  0.0335, -1.3948, -1.4061],
          [-0.7371,  0.0523,  0.6766,  ..., -1.0495,  0.2992,  0.0780]]]])
torch.Size([1, 1, 32, 32])
tensor([[ 0.0858, -0.0585, -0.1606,  0.1272,  0.0304, -0.0677, -0.0085,  0.1100,
         -0.0551, -0.0508]], grad_fn=<ThAddmmBackward>)


In [11]:
## 全てのパラメータとバックプロットのグラディエントバッファをゼロにする
net.zero_grad()
out.backward(torch.randn(1,10))

<h2>使用したクラス</h2>
<h3>torch.Tensor()</h3>
<h5>多次元配列の定義</h5>
<h3>backward()</h3>
<h5></h5>
<h3>nn.Module</h3>
<h5>ニューラルネットワークモジュール。パラメータをカプセル化する便利な方法</h5>
<h3>nn.Parameter</h3>
<h5>Tensorの一種。属性として割り当てられた時にパラメータとして自動的に登録される</h5>
<h3>autograd.Function</h3>
<h5>forward()およびbackward()の定義を実装。</h5>

In [13]:
## 損失の計算
## nnパッケージにいくつかの損失関数がある
## 代表的なものはnn.MSELoss <- 入力とターゲット間の平均二乗誤差を計算

# 入力データをネットワークに入力し、出力を得る
output = net(input)
print(output.size())
# ダミーの正解データ（target）を作成
target = torch.randn(10)
print(target.size())
# ダミーターゲットのリサイズ
target = target.view(1,-1)
print(target.size())
# 損失関数の定義
criterion = nn.MSELoss()
# 出力とダミーターゲットのロスを求める
loss = criterion(output,target)
print(loss)

torch.Size([1, 10])
torch.Size([10])
torch.Size([1, 10])
tensor(2.0183, grad_fn=<MseLossBackward>)


<h2>backwardでは次のような計算のグラフが表示される</h2>
<h3>input -> conv2 -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d</h3>
<h3>-> view -> linear ->relu -> linear -> relu -> linear </h3>
<h3>-> MSELodd</h3>
<h3>-> loss</h3>

In [15]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) #Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #ReLU

In [16]:
net.zero_grad() #全てのパラメータのグラディエントバッファをゼロにする
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0050, -0.0178,  0.0331,  0.0107, -0.0277,  0.0197])


In [17]:
## ネットワークの重み更新
## 代表的な更新ルールは確率的勾配降下法(SGD)
## weight = weight - learning_rate * gradient
## 無理やり実装すると以下
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

<h4>ニューラルネットワークを使用する場合は、SGD、Nesterov-SGD、Adam、RMSPropなどの様々なことなる更新ルールを使用する必要がある。これを可能にするために、torch.optimを使用する</h4>

In [18]:
import torch.optim as optim
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)
# in your training loop:
# 勾配が蓄積されているため、勾配バッファは手動でゼロにする必要がある
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() #重みの更新